In [1]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import ChatterBoxBackend
from pylabrobot.visualizer.visualizer import Visualizer
from pylabrobot.resources.opentrons import OTDeck

from pylabrobot.resources.opentrons.load import *
from pylabrobot.resources.opentrons.plates import *

from pylabrobot.resources import set_tip_tracking, set_volume_tracking#, set_cross_contamination_tracking
set_tip_tracking(True), set_volume_tracking(True)

# Optional
#set_cross_contamination_tracking(True)

import opentrons
import time

lh = LiquidHandler(backend=ChatterBoxBackend(), deck=OTDeck())

await lh.setup()

vis = Visualizer(resource=lh)
await vis.setup()

from pylabrobot.resources import (
    opentrons_24_tuberack_eppendorf_2ml_safelock_snapcap,
    opentrons_96_tiprack_300ul,
    agilent_1_reservoir_290ml,
    nest_12_reservoir_15ml
)

# When you instantiate a labware, give it a name that will show when you
# mouse over it in the visualizer.
tip_rack_1 = opentrons_96_tiprack_300ul("tip_rack_1")
tip_rack_2 = opentrons_96_tiprack_300ul("tip_rack_2")
tip_rack_3 = opentrons_96_tiprack_300ul("tip_rack_3")
tip_rack_4 = opentrons_96_tiprack_300ul("tip_rack_4")

lh.deck.assign_child_at_slot(tip_rack_1, 1)
lh.deck.assign_child_at_slot(tip_rack_2, 2)
lh.deck.assign_child_at_slot(tip_rack_3, 4)
lh.deck.assign_child_at_slot(tip_rack_4, 5)

# Wash Buffer Stock
#wash_stock = agilent_1_reservoir_290ml("wash_stock")
wash_stock = load_opentrons_resource("../../Teaching Lab Stuff/vwr_96_wellplate_2200ul.json", "wash_stock")

lh.deck.assign_child_at_slot(wash_stock, 3)

# Liquid Waste
#liquid_waste = agilent_1_reservoir_290ml("liquid_waste")
liquid_waste = load_opentrons_resource("../../Teaching Lab Stuff/vwr_96_wellplate_2200ul.json", "liquid_waste")

lh.deck.assign_child_at_slot(liquid_waste, 9)

# Dummy reservoir
#dummy = nest_12_reservoir_15ml("dummy")
#lh.deck.assign_child_at_slot(dummy, 10)

# ELISA Plate
#prep_plate = corning_96_wellplate_360ul_flat("prep_plate")
prep_plate = load_opentrons_resource("../../Teaching Lab Stuff/greiner_96_wellplate_340ul.json", "prep_plate")
lh.deck.assign_child_at_slot(prep_plate, 6)

wash_stock.set_well_liquids((f"Wash_Buffer", 2000))

rack_1_walker = tip_rack_1.traverse(batch_size=8, direction='down')
rack_2_walker = tip_rack_2.traverse(batch_size=8, direction='down')
rack_3_walker = tip_rack_3.traverse(batch_size=8, direction='down')
rack_4_walker = tip_rack_4.traverse(batch_size=8, direction='down')

tip_walkers = [rack_1_walker, rack_2_walker,
               rack_3_walker, rack_4_walker]

cur_walker = tip_walkers[0]
cur_walker_spot = 0

Setting up the robot.
Resource deck was assigned to the robot.
Resource trash_container was assigned to the robot.
Websocket server started at http://127.0.0.1:2121
File server started at http://127.0.0.1:1337 . Open this URL in your browser.
Resource tip_rack_1 was assigned to the robot.
Resource tip_rack_2 was assigned to the robot.
Resource tip_rack_3 was assigned to the robot.
Resource tip_rack_4 was assigned to the robot.
Resource wash_stock was assigned to the robot.
Resource liquid_waste was assigned to the robot.
Resource prep_plate was assigned to the robot.


In [3]:
async def do_wash(left_col, right_col, tip_walkers):

    cur_walker = tip_walkers[0]
    cur_walker_spot = 0
    
    for i in range(left_col, right_col+1):
        col_num = i

        try:
            tips = next(cur_walker)
        except:
            print("Out of tips in this rack!")
            cur_walker_spot += 1
            cur_walker = tip_walkers[cur_walker_spot]
            try:
                tips = next(cur_walker)
            except:
                print("Yikes")

        await lh.pick_up_tips(tips)
        
        # Move wash from stock to plate
        await lh.aspirate(wash_stock[f'A{col_num}:H{col_num}'], vols=[200]*8)
        await lh.dispense(prep_plate[f'A{col_num}:H{col_num}'], vols=[200]*8)
        #time.sleep(0.5)

        # Mix in place
        num_mix = 3

        for _ in range(num_mix):
            #time.sleep(0.25)
            await lh.aspirate(prep_plate[f'A{col_num}:H{col_num}'], vols=[75]*8)
            #time.sleep(0.25)
            await lh.dispense(prep_plate[f'A{col_num}:H{col_num}'], vols=[75]*8)
            
        #time.sleep(0.5)
    
        # Move wash from plate to liquid trash
        await lh.aspirate(prep_plate[f'A{col_num}:H{col_num}'], vols=[200]*8)
        await lh.dispense(liquid_waste[f'A{col_num}:H{col_num}'], vols=[200]*8)
        #time.sleep(0.5)

        await lh.discard_tips()
    
    print("Job's Done!")

In [4]:
print("Starting wash 1:\n\n")
await do_wash(left_col=1, right_col=12, tip_walkers=tip_walkers)

Starting wash 1:


Picking up tips [Pickup(resource=TipSpot(name=tip_rack_1_A1, location=(012.531, 072.391, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=Coordinate(x=0, y=0, z=0), tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(resource=TipSpot(name=tip_rack_1_B1, location=(012.531, 063.391, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=Coordinate(x=0, y=0, z=0), tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(resource=TipSpot(name=tip_rack_1_C1, location=(012.531, 054.391, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=Coordinate(x=0, y=0, z=0), tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(resource=TipSpot(name=tip_rack_1_D1, location=(012.531, 045.391, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=Coordinate(x=0, 

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 48192)
Traceback (most recent call last):
  File "/usr/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/mnt/d/Chory Lab/PyLabRobot/pylabrobot/visualizer/visualizer.py", line 322, in __init__
    super().__init__(*args, directory=path, **kwargs)
  File "/usr/lib/python3.10/http/server.py", line 651, in __init__
    super().__init__(*args, **kwargs)
  File "/usr/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/lib/python3.10/http/server.py", line 425, in handle
    self.handle_one_request()
  File "/usr/lib/python

In [2]:
print("Starting wash 1:\n\n")
await do_wash(left_col=1, right_col=12, tip_walkers=tip_walkers)

print("Starting wash 2:\n\n")
await do_wash(left_col=3, right_col=12, tip_walkers=tip_walkers)
print("Starting wash 3:\n\n")
await do_wash(left_col=5, right_col=12, tip_walkers=tip_walkers)
print("Starting wash 4:\n\n")
await do_wash(left_col=7, right_col=12, tip_walkers=tip_walkers)
print("Starting wash 5:\n\n")
await do_wash(left_col=9, right_col=12, tip_walkers=tip_walkers)
print("Starting wash 6:\n\n")
await do_wash(left_col=11, right_col=12, tip_walkers=tip_walkers)


Starting wash 1:


Picking up tips [Pickup(resource=TipSpot(name=tip_rack_1_A1, location=(014.380, 074.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(resource=TipSpot(name=tip_rack_1_B1, location=(014.380, 065.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(resource=TipSpot(name=tip_rack_1_C1, location=(014.380, 056.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(resource=TipSpot(name=tip_rack_1_D1, location=(014.380, 047.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fi

UnboundLocalError: local variable 'tips' referenced before assignment

### Imports

In [1]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import ChatterBoxBackend
from pylabrobot.visualizer.visualizer import Visualizer
from pylabrobot.resources.opentrons import OTDeck

from pylabrobot.resources.opentrons.load import *
from pylabrobot.resources.opentrons.plates import *

from pylabrobot.resources import set_tip_tracking, set_volume_tracking#, set_cross_contamination_tracking
set_tip_tracking(True), set_volume_tracking(True)

# Optional
#set_cross_contamination_tracking(True)

import opentrons
import time

### Set up the deck

In [2]:
lh = LiquidHandler(backend=ChatterBoxBackend(), deck=OTDeck())

await lh.setup()

vis = Visualizer(resource=lh)
await vis.setup()

Setting up the robot.
Resource deck was assigned to the robot.
Resource trash_container was assigned to the robot.
Websocket server started at http://127.0.0.1:2122
File server started at http://127.0.0.1:1338 . Open this URL in your browser.


### Import necessary labware

In [3]:
from pylabrobot.resources import (
    opentrons_24_tuberack_eppendorf_2ml_safelock_snapcap,
    opentrons_96_tiprack_300ul,
    agilent_1_reservoir_290ml,
    nest_12_reservoir_15ml
)

### Put Labware on Deck

In [4]:
# When you instantiate a labware, give it a name that will show when you
# mouse over it in the visualizer.
tip_rack_1 = opentrons_96_tiprack_300ul("tip_rack_1")
tip_rack_2 = opentrons_96_tiprack_300ul("tip_rack_2")
tip_rack_3 = opentrons_96_tiprack_300ul("tip_rack_3")
tip_rack_4 = opentrons_96_tiprack_300ul("tip_rack_4")

lh.deck.assign_child_at_slot(tip_rack_1, 1)
lh.deck.assign_child_at_slot(tip_rack_2, 2)
lh.deck.assign_child_at_slot(tip_rack_3, 4)
lh.deck.assign_child_at_slot(tip_rack_4, 5)

# Wash Buffer Stock
#wash_stock = agilent_1_reservoir_290ml("wash_stock")
wash_stock = load_opentrons_resource("../../Teaching Lab Stuff/vwr_96_wellplate_2200ul.json", "wash_stock")

lh.deck.assign_child_at_slot(wash_stock, 3)

# Liquid Waste
#liquid_waste = agilent_1_reservoir_290ml("liquid_waste")
liquid_waste = load_opentrons_resource("../../Teaching Lab Stuff/vwr_96_wellplate_2200ul.json", "liquid_waste")

lh.deck.assign_child_at_slot(liquid_waste, 9)

# Dummy reservoir
#dummy = nest_12_reservoir_15ml("dummy")
#lh.deck.assign_child_at_slot(dummy, 10)

# ELISA Plate
#prep_plate = corning_96_wellplate_360ul_flat("prep_plate")
prep_plate = load_opentrons_resource("../../Teaching Lab Stuff/greiner_96_wellplate_340ul.json", "prep_plate")
lh.deck.assign_child_at_slot(prep_plate, 6)

Resource tip_rack_1 was assigned to the robot.
Resource tip_rack_2 was assigned to the robot.
Resource tip_rack_3 was assigned to the robot.
Resource tip_rack_4 was assigned to the robot.
Resource wash_stock was assigned to the robot.
Resource liquid_waste was assigned to the robot.
Resource prep_plate was assigned to the robot.


In [7]:
tip_rack_1.fill()

### Fill Labware with Liquids

In [5]:
wash_stock.set_well_liquids((f"Wash_Buffer", 2000))

### Helper functions for protocol

In [6]:
def do_wash(buffer_location, ELISA_wells, ELISA_plate, liquid_waste, pipette, wash_volume =300):
    """
    @param buffer_location: (opentrons.labware) object containing the wash buffer
    @param ELISA_wells: (list) list of wells to dispense buffer in to
    @param ELISA_plate: (opentrons.labware) object representing ELISA plate
    @param liquid_waste: (opentrons.labware) object representing the waste plate
    @param pipette: (opentrons.pipette) object that is the pipette to use for this protocol
    @param wash_volume: (int) volume in uL to wash with
   
    This function will aspirate 'wash_volume' uL of wash buffer from 'buffer_location', dispense it to each
    column in source_wells, aspirate from the ELISA plate, discard liquid wash in waste, and trash the tips
    """
    for well in ELISA_wells:   
       pipette.pick_up_tip()
       # Add buffer to plate
       pipette.transfer(wash_volume, buffer_location[well], ELISA_plate.wells(well),
                        mix_after=(3, 75), new_tip = 'never')  # mix 3 times with 75uL after dispensing
       # Remove and discard wash buffer
       pipette.transfer(wash_volume, ELISA_plate.wells(well), liquid_waste[well], new_tip = 'never')
       # trash tip
       pipette.drop_tip()

### Scratch Work

In [6]:
row_A = ['A1', 'A2', 'A3', 'A4',
         'A5', 'A6', 'A7', 'A8',
         'A9', 'A10', 'A11', 'A12']

In [6]:
rack_1_walker = tip_rack_1.traverse(batch_size=8, direction='down')
rack_2_walker = tip_rack_2.traverse(batch_size=8, direction='down')
rack_3_walker = tip_rack_3.traverse(batch_size=8, direction='down')
rack_4_walker = tip_rack_4.traverse(batch_size=8, direction='down')

tip_walkers = [rack_1_walker, rack_2_walker,
               rack_3_walker, rack_4_walker]

cur_walker = tip_walkers[0]
cur_walker_spot = 0

async def do_wash(left_col, right_col, tip_walkers):

    cur_walker = tip_walkers[0]
    cur_walker_spot = 0
    
    for i in range(left_col, right_col+1):
        col_num = i

        try:
            tips = next(cur_walker)
        except:
            print("Out of tips in this rack!")
            cur_walker_spot += 1
            cur_walker = tip_walkers[cur_walker_spot]
            try:
                tips = next(cur_walker)
            except:
                print("Yikes")

        await lh.pick_up_tips(tips)
        
        # Move wash from stock to plate
        await lh.aspirate(wash_stock[f'A{col_num}:H{col_num}'], vols=200)
        await lh.dispense(prep_plate[f'A{col_num}:H{col_num}'], vols=200)
        time.sleep(0.5)

        # Mix in place
        num_mix = 3

        for _ in range(num_mix):
            time.sleep(0.25)
            await lh.aspirate(prep_plate[f'A{col_num}:H{col_num}'], vols=75)
            time.sleep(0.25)
            await lh.dispense(prep_plate[f'A{col_num}:H{col_num}'], vols=75)
            
        time.sleep(0.5)
    
        # Move wash from plate to liquid trash
        await lh.aspirate(prep_plate[f'A{col_num}:H{col_num}'], vols=200)
        await lh.dispense(liquid_waste[f'A{col_num}:H{col_num}'], vols=200)
        time.sleep(0.5)

        await lh.discard_tips()
    
    print("Job's Done!")

In [7]:
print("Starting wash 1:\n\n")
await do_wash(left_col=1, right_col=12, tip_walkers=tip_walkers)
print("Starting wash 2:\n\n")
await do_wash(left_col=3, right_col=12, tip_walkers=tip_walkers)
print("Starting wash 3:\n\n")
await do_wash(left_col=5, right_col=12, tip_walkers=tip_walkers)
print("Starting wash 4:\n\n")
await do_wash(left_col=7, right_col=12, tip_walkers=tip_walkers)
print("Starting wash 5:\n\n")
await do_wash(left_col=9, right_col=12, tip_walkers=tip_walkers)
print("Starting wash 6:\n\n")
await do_wash(left_col=11, right_col=12, tip_walkers=tip_walkers)

Starting wash 1:


Picking up tips [Pickup(resource=TipSpot(name=tip_rack_1_A1, location=(014.380, 074.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(resource=TipSpot(name=tip_rack_1_B1, location=(014.380, 065.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(resource=TipSpot(name=tip_rack_1_C1, location=(014.380, 056.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(resource=TipSpot(name=tip_rack_1_D1, location=(014.380, 047.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fi

UnboundLocalError: local variable 'tips' referenced before assignment

In [62]:
# Runs one cycle of washing for whole 96 plate

left_col = 1
right_col = 12

for i in range(left_col, right_col+1):
    col_num = i

    try:
        tips = next(cur_walker)
    except:
        print("Out of tips in this rack!")
        cur_walker_spot += 1
        cur_walker = tip_walkers[cur_walker_spot]
        try:
            tips = next(cur_walker)
        except:
            print("Yikes")

    await lh.pick_up_tips(tips)
    
    # Move wash from stock to plate
    await lh.aspirate(wash_stock[f'A{col_num}:H{col_num}'], vols=200)
    await lh.dispense(prep_plate[f'A{col_num}:H{col_num}'], vols=200)
    time.sleep(0.3)

    # Mix in place
    num_mix = 3

    for _ in range(num_mix):
        time.sleep(0.1)
        await lh.aspirate(prep_plate[f'A{col_num}:H{col_num}'], vols=75)
        time.sleep(0.1)
        await lh.dispense(prep_plate[f'A{col_num}:H{col_num}'], vols=75)
    time.sleep(0.3)
    
    # Move wash from plate to liquid trash
    await lh.aspirate(prep_plate[f'A{col_num}:H{col_num}'], vols=200)
    await lh.dispense(liquid_waste[f'A{col_num}:H{col_num}'], vols=200)
    time.sleep(0.3)

    await lh.discard_tips()
    


Out of tips in this rack!
Picking up tips [Pickup(resource=TipSpot(name=tip_rack_4_A1, location=(014.380, 074.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(resource=TipSpot(name=tip_rack_4_B1, location=(014.380, 065.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(resource=TipSpot(name=tip_rack_4_C1, location=(014.380, 056.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=300.0, fitting_depth=7.47)), Pickup(resource=TipSpot(name=tip_rack_4_D1, location=(014.380, 047.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot), offset=None, tip=Tip(has_filter=False, total_tip_length=59.3, maximal_volume=30

In [59]:
tips

[TipSpot(name=tip_rack_1_A12, location=(113.380, 074.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot),
 TipSpot(name=tip_rack_1_B12, location=(113.380, 065.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot),
 TipSpot(name=tip_rack_1_C12, location=(113.380, 056.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot),
 TipSpot(name=tip_rack_1_D12, location=(113.380, 047.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot),
 TipSpot(name=tip_rack_1_E12, location=(113.380, 038.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot),
 TipSpot(name=tip_rack_1_F12, location=(113.380, 029.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot),
 TipSpot(name=tip_rack_1_G12, location=(113.380, 020.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip_spot),
 TipSpot(name=tip_rack_1_H12, location=(113.380, 011.240, 005.390), size_x=3.698, size_y=3.698, size_z=0, category=tip

In [41]:
cur_walker_spot = 0
cur_walker = tip_walkers[cur_walker_spot]
try:
    next(cur_walker)
except:
    print("Out of tips in this rack!")
    cur_walker_spot += 1
    cur_walker = tip_walkers[cur_walker_spot]
    try:
        next(cur_walker)
    except:
        print("Yikes")

NameError: name 'tip_walkers' is not defined